In [1]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

# from models.keras_ssd512 import ssd_512
from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

Using TensorFlow backend.


In [2]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 1 # Number of positive classes, e.g. 20 for Pascal VOC, 80 for MS COCO
scales_pascal = [0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05] # The anchor box scaling factors used in the original SSD300 for the Pascal VOC datasets
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_pascal
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True

In [3]:
# 1: Build the Keras model.

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)

# model = ssd_512(image_size=(img_height, img_width, 3),
#                 n_classes=20,
#                 mode='inference',
#                 l2_regularization=0.0005,
#                 scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
#                 aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
#                                          [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                          [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                          [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                          [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
#                                          [1.0, 2.0, 0.5],
#                                          [1.0, 2.0, 0.5]],
#                two_boxes_for_ar1=True,
#                steps=[8, 16, 32, 64, 128, 256, 512],
#                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
#                clip_boxes=False,
#                variances=[0.1, 0.1, 0.2, 0.2],
#                normalize_coords=True,
#                subtract_mean=[123, 117, 104],
#                swap_channels=[2, 1, 0],
#                confidence_thresh=0.5,
#                iou_threshold=0.45,
#                top_k=200,
#                nms_max_output_size=400)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
weights_path = 'VGG_ILSVRC_16_layers_fc_reduced.h5'

model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

# adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

# model.compile(optimizer=sgd, loss=ssd_loss.compute_loss, metrics=['accuracy'])
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
images_dir      = 'GreenParking/'
annotations_dir      = 'GreenParking/'
train_image_set_filename    = 'GreenParking/train.txt'

classes = ['plate']


train_dataset.parse_xml(images_dirs=[images_dir],
                        image_set_filenames=[train_image_set_filename],
                        annotations_dirs=[annotations_dir],
                        classes=classes,
                        include_classes='all',
                        exclude_truncated=False,
                        exclude_difficult=False,
                        ret=False)

train_dataset.create_hdf5_dataset(file_path='dataset_plate_trainval.h5',
                                  resize=False,
                                  variable_image_size=True,
                                  verbose=True)

Creating HDF5 dataset: 100%|██████████| 1647/1647 [00:10<00:00, 150.07it/s]


In [5]:
# 3: Set the batch size.

batch_size = 8 # Change the batch size if you like, or if you run into GPU memory issues.

# 4: Set the image transformations for pre-processing and data augmentation options.

# For the training generator:
ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# 5: Instantiate an encoder that can encode ground truth labels into the format needed by the SSD loss function.

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

print(n_classes)
print(len(classes))

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.

train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[ssd_data_augmentation],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)


# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))

1
1
Number of images in the training dataset:	  1647


In [6]:
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001

In [7]:
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='ssd300_plate_epoch-{epoch:02d}_loss-{loss:.4f}_acc-{acc:.3f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 

csv_logger = CSVLogger(filename='ssd300_plate_training_log.csv',
                       separator=',',
                       append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
                                                verbose=1)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             learning_rate_scheduler,
             terminate_on_nan]

In [8]:
#### If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 30
steps_per_epoch = 300

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              callbacks=callbacks,
                              epochs=final_epoch,
                              initial_epoch=initial_epoch)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100


 204/1000 [=====>........................] - ETA: 5:06:32 - loss: 2.8816 - acc: 0.08 - ETA: 2:38:56 - loss: 2.8816 - acc: 0.09 - ETA: 1:49:34 - loss: 2.8816 - acc: 0.08 - ETA: 1:24:54 - loss: 2.8816 - acc: 0.08 - ETA: 1:10:06 - loss: 2.8816 - acc: 0.08 - ETA: 1:00:13 - loss: 2.8816 - acc: 0.08 - ETA: 53:09 - loss: 2.8816 - acc: 0.0845 - ETA: 47:52 - loss: 2.8816 - acc: 0.08 - ETA: 43:46 - loss: 2.8816 - acc: 0.08 - ETA: 40:28 - loss: 2.8815 - acc: 0.08 - ETA: 37:47 - loss: 2.8815 - acc: 0.08 - ETA: 35:32 - loss: 2.8815 - acc: 0.08 - ETA: 33:38 - loss: 2.8815 - acc: 0.08 - ETA: 32:00 - loss: 2.8815 - acc: 0.08 - ETA: 30:35 - loss: 2.8815 - acc: 0.08 - ETA: 29:21 - loss: 2.8814 - acc: 0.08 - ETA: 28:15 - loss: 2.8814 - acc: 0.08 - ETA: 27:16 - loss: 2.8814 - acc: 0.08 - ETA: 26:23 - loss: 2.8814 - acc: 0.08 - ETA: 25:35 - loss: 2.8814 - acc: 0.08 - ETA: 24:52 - loss: 2.8813 - acc: 0.08 - ETA: 24:13 - loss: 2.8813 - acc: 0.08 - ETA: 23:37 - loss: 2.8813 - acc: 0.08 - ETA: 23:04 - loss: 2.

KeyboardInterrupt: 